In [ ]:
# ! pip install pytorch-ignite

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
torch.cuda.empty_cache()

import torchvision
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.backends.cudnn as cudnn

from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from timeit import default_timer as timer 

%matplotlib inline
from matplotlib import pyplot as plt

import sys
sys.path.append('/content/drive/MyDrive/NN_Course_Project/project/lib')

from data import loader
from train_test import TrainTestModel
from helper_funcs import MetricsComputation, select_model, print_decos, select_loss_func
from m_vgg16 import VGG16

from ignite.engine import Engine, Events
from ignite.handlers import Checkpoint, global_step_from_engine

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Baseline loss function
loss_fn = select_loss_func('baseline')
ranking_criterion = nn.MarginRankingLoss(margin=0.0)
learning_rate = 0.1

'cuda'

# CIFAR-10 Dataset

In [ ]:
# dataloaders for datasets
dataloader_train, dataloader_test = loader('CIFAR10', batch_size=128)

Files already downloaded and verified
Files already downloaded and verified
Length of train dataloader: 391 batches of 128
Length of test dataloader: 79 batches of 128


#### VGG16 Model

In [ ]:
M_VGG16   = VGG16(num_classes=10) 
optimizer = optim.SGD(M_VGG16.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001, nesterov=False)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[10, 150, 250], gamma=0.1) # Decays learning rate

#### Training

In [ ]:
print_decos('train')
start_time = timer()

VGG_C10 = TrainTestModel()

# Training from the begining
train_results = VGG_C10.train(model=M_VGG16,
                        model_name='vgg16',
                        dataset='cifar10',
                        train_dataloader=dataloader_train, 
                        loss_fn= loss_fn, 
                        optimizer=optimizer,
                        scheduler=scheduler,
                        epochs=300, 
                        device=device) 


end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

TRAINING INITIATED



  0%|          | 0/300 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.3003 | train_acc: 11.0270% | 
Epoch: 2 | train_loss: 2.2986 | train_acc: 11.7339% | 
Epoch: 3 | train_loss: 2.2978 | train_acc: 11.6105% | 
Epoch: 4 | train_loss: 2.2974 | train_acc: 11.8103% | 
Epoch: 5 | train_loss: 2.2971 | train_acc: 11.4730% | 
Epoch: 6 | train_loss: 2.2963 | train_acc: 11.8398% | 
Epoch: 7 | train_loss: 2.2955 | train_acc: 11.7527% | 
Epoch: 8 | train_loss: 2.2950 | train_acc: 11.7188% | 
Epoch: 9 | train_loss: 2.2938 | train_acc: 11.4990% | 
Epoch: 10 | train_loss: 2.2925 | train_acc: 11.7016% | 
Epoch: 11 | train_loss: 2.2909 | train_acc: 11.8386% | 
Epoch: 12 | train_loss: 2.2887 | train_acc: 11.6820% | 
Epoch: 13 | train_loss: 2.2853 | train_acc: 11.6972% | 
Epoch: 14 | train_loss: 2.2803 | train_acc: 12.1771% | 
Epoch: 15 | train_loss: 2.2733 | train_acc: 12.4389% | 
Epoch: 16 | train_loss: 2.2618 | train_acc: 12.8513% | 
Epoch: 17 | train_loss: 2.2417 | train_acc: 13.8795% | 
Epoch: 18 | train_loss: 2.2064 | train_acc: 15.6622% | 
E

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/NN_Course_Project/project/models'
checkpoint = torch.load(os.path.join(MODEL_PATH, 'vgg16', 'vgg16_cifar10_200.pt'))
M_VGG16   = VGG16(num_classes=10)
M_VGG16.load_state_dict(checkpoint[])


# Resume training
# results = VGG_C10.train(model=M_VGG16,
#                         model_name='vgg16',
#                         dataset='cifar10',
#                         train_dataloader=dataloader_train, 
#                         loss_fn= loss_fn, 
#                         optimizer=optimizer,
#                         scheduler=scheduler,
#                         epochs=8, 
#                         device=device, 
#                         resume_epoch=5, 
#                         resume_model_pth= checkpoint_pth) 
# print(results)

{'milestones': Counter({10: 1, 150: 1, 250: 1}), 'gamma': 0.1, 'base_lrs': [0.1], 'last_epoch': 1955, 'verbose': False, '_step_count': 1956, '_get_lr_called_within_step': False, '_last_lr': [0.00010000000000000003]}


#### Testing

In [ ]:
print_decos('eval')

test_results = VGG_C10.test(model=M_VGG16, 
                            dataloader=dataloader_test, 
                            loss_fn=loss_fn, 
                            device=device)


VGG_C10_Metrics = MetricsComputation(test_results)
aurc, eaurc, fpr, fpr_in_tpr_95, pr_auc = VGG_C10_Metrics.compute_metrics()

print(f"Test Accuracy: {test_results[1]}")
print(f"Area Under Risk Curve (AURC): {aurc}")
print(f"Excessive-AURC (E-AURC): {eaurc}")
print(f"Area Under Precision-Recall Curve (AUPR): {pr_auc}")
print(f"False Positive Rate (FPR) at 95% True Positive Rate (TPR): {fpr_in_tpr_95}\n")

#### PreAct-ResNet110 Model

In [ ]:
# model = select_model()
M_VGG16 = VGG16(num_classes=100) 

optimizer = optim.SGD(M_VGG16.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001, nesterov=False)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[10, 150, 250], gamma=0.1) # Decays learning rate

# dataloaders for datasets
dataloader_train, dataloader_test = loader('CIFAR100', batch_size=128)

start_time = timer()

VGG_C10 = TrainTestModel()
results = VGG_C10.train(model=M_VGG16, 
                        train_dataloader=dataloader_train, 
                        loss_fn= loss_fn, 
                        optimizer=optimizer,
                        scheduler=scheduler,
                        epochs=300, 
                        device=device) 

end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

print("\n========================================================================================\n")

test_loss, test_acc, test_preds, labels = VGG_C10.test(model=M_VGG16, 
                                                       dataloader=dataloader_test, 
                                                       loss_fn=loss_fn, 
                                                       device=device)

VGG_C10_Metrics = MetricsComputation(test_preds, labels)
aurc, eaurc, pr_auc, fpr_in_tpr_95 = VGG_C10_Metrics.compute_metrics()

print(f"Area Under Risk Curve (AURC): {aurc}")
print(f"Excessive-AURC (E-AURC): {eaurc}")
print(f"Area Under Precision-Recall Curve (AUPR): {pr_auc}")
print(f"False Positive Rate (FPR) at 95% True Positive Rate (TPR): {fpr_in_tpr_95}\n")

# Save model
save_model('vgg16_cifar100.pth', M_VGG16)

#### DenseNet-BC Model

# CIFAR100 Dataset
#### VGG16 Model